In [1]:
# Importing necessary library
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pywt
import pandas as pd
import os
import time
from scipy import signal
from stingray import lightcurve
import sys
from stingray import Bispectrum
import warnings
import csv
warnings.filterwarnings('ignore')
%matplotlib inline

c:\Users\fulky\anaconda3\envs\skripsi\lib\site-packages\stingray\utils.py:25: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn("pyfftw not installed. Using standard scipy fft")
c:\Users\fulky\anaconda3\envs\skripsi\lib\site-packages\stingray\utils.py:40: UserWarning: Numba not installed. Faking it
  warnings.warn("Numba not installed. Faking it")


In [2]:
# file that will be skipped because of information loss
zeros_test = ['co2a0000368_91.csv', 'co2c0000341_26.csv']
zeros_train = ['co2a0000368_0.csv', 'co2a0000368_1.csv', 'co2a0000368_2.csv', 'co2a0000368_3.csv', 'co2a0000368_4.csv', 'co2a0000368_5.csv', 'co2c0000341_27.csv']

In [3]:
def calcBispecMag(df_data, t, lag):
    # Compute the bispectrum of the signal
    lc = lightcurve.Lightcurve(t,df_data.T)
    bs = Bispectrum(lc, maxlag=lag)

    # Plot the bispectrum using contour plots
    # plt.contour(bs.freq, bs.freq, bs.bispec_mag)
    # plt.xlabel('f1')
    # plt.ylabel('f2')
    # plt.show()

    # Plot the bispectrum using mesh plots
    # fig = plt.figure()
    # ax = fig.add_subplot(111, projection='3d')
    # X, Y = np.meshgrid(bs.freq, bs.freq)
    # ax.plot_surface(X, Y, bs.bispec_mag)
    # ax.set_xlabel('f1')
    # ax.set_ylabel('f2')
    # ax.set_zlabel('Bispectrum')
    # plt.show()

    return bs.bispec_mag

In [4]:
def gkernel(k=2, l=1, sig=20):
    """
    Gaussian Kernel Creator via given length and sigma
    """

    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    bx = np.linspace(-(k - 1) / 2., (k - 1) / 2., k)

    xx, yy = np.meshgrid(ax, bx)

    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sig))

    return kernel / np.max(kernel)

In [5]:
def create_feature(bispectrum, pyramid, fil_size):
    fsize = len(fil_size)
    feature_matrix = np.zeros([fsize,fsize])
    xtrack = 0
    ytrack = 0
    for xdim in range(fsize):
        for ydim in range(fsize):
            x,y = fil_size[xdim], fil_size[ydim]
            # print(xtrack, ytrack)
            
            feature_matrix[xdim][ydim] = np.mean(bispectrum[xtrack:xtrack+x, ytrack:ytrack+y] * pyramid[xtrack:xtrack+x, ytrack:ytrack+y])
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    final_feature = feature_matrix[np.triu(np.ones_like(feature_matrix, dtype=bool))]
    final_feature = final_feature[~np.isnan(final_feature)]

    return feature_matrix

    # x, y = np.meshgrid(np.arange(256), np.arange(256))
    # z = overall
    # z = z[:-1, :-1]
    # z_min, z_max = -np.abs(z).max(), np.abs(z).max()

    # plt.subplot()

    # plt.pcolormesh(x, y, z, 
    #             cmap =cm.coolwarm, 
    #             vmin = z_min, 
    #             vmax = z_max,
    #             edgecolors = 'face',
    #             shading ='flat')

    # plt.title('Filter')

    # # set the limits of the plot
    # # to the limits of the data
    # plt.axis([x.min(), x.max(), y.min(), y.max()])

    # plt.colorbar()
    # plt.show()

    # fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 5))
    # ax = plt.axes(projection="3d")
    # x2d, y2d = np.meshgrid(np.arange(256), np.arange(256))
    # ax.plot_surface(x2d, y2d, overall, cmap=cm.coolwarm,
    #                         linewidth=0, antialiased=False)
    # ax.set_xlabel('x')
    # ax.set_ylabel('y')
    # ax.set_zlabel('height')
    # plt.show()

In [6]:
# Define sampling frequency
fs = 256
t = np.arange(0, 1, 1/fs)

def get_csv_EEG(filename):
    # Load data from CSV
    data = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=range(3,259))
    channel_name = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=1, dtype='str', encoding='utf-8')
    
    df_data = pd.DataFrame(data.T, columns=channel_name)

    df_data = df_data.drop(columns=['X', 'Y', 'nd'])

    return df_data, df_data.columns


In [7]:
def calcCumulantOrde3(df_data, t, lag):
    # Compute the bispectrum of the signal
    lc = lightcurve.Lightcurve(t,df_data.T)
    bs = Bispectrum(lc, maxlag=lag)

    # Plot the bispectrum using contour plots
    # plt.contour(bs.freq, bs.freq, bs.bispec_mag)
    # plt.xlabel('f1')
    # plt.ylabel('f2')
    # plt.show()

    # # Plot the bispectrum using mesh plots
    # fig = plt.figure()
    # ax = fig.add_subplot(111, projection='3d')
    # X, Y = np.meshgrid(bs.freq, bs.freq)
    # ax.plot_surface(X, Y, bs.bispec_mag)
    # ax.set_xlabel('f1')
    # ax.set_ylabel('f2')
    # ax.set_zlabel('Bispectrum')
    # plt.show()

    return bs

In [8]:
def calcWaveletDec(bs):
    # Select wavelet and decomposition level
    wavelet = 'db4'
    level = 5

    # Deecompose signal
    coeffs = pywt.wavedec2(bs.cum3, wavelet, level=level)

    # print(coeffs)
    # Visualize
    
    cA5 = coeffs[0][np.triu(np.ones_like(coeffs[0], dtype=bool))]
    cD5 = np.ravel([coeffs[1][0], coeffs[1][1], coeffs[1][2]])
    cD4 = np.ravel([coeffs[2][0], coeffs[2][1], coeffs[2][2]])
    cD3 = np.ravel([coeffs[3][0], coeffs[3][1], coeffs[3][2]])
    cD2 = np.ravel([coeffs[4][0], coeffs[4][1], coeffs[4][2]])
    cD1 = np.ravel([coeffs[5][0], coeffs[5][1], coeffs[5][2]])
    
    coeff = [cA5,cD5,cD4,cD3,cD2,cD1]
    # fig, axs = plt.subplots(6)
    
    
    # axs[0].plot(cA5)
    # axs[0].set_title(f'Approximation - Level 5')
    # axs[1].plot(cD5)
    # axs[1].set_title(f'Detail - Level 5')
    # axs[2].plot(cD4)
    # axs[2].set_title(f'Detail - Level 4')
    # axs[3].plot(cD3)
    # axs[3].set_title(f'Detail - Level 3')
    # axs[4].plot(cD2)
    # axs[4].set_title(f'Detail - Level 2')
    # axs[5].plot(cD1)
    # axs[5].set_title(f'Detail - Level 1')
    # plt.show()

    return coeffs


In [9]:
def calcRelativeEnergy(coeffs, df_data):
    # Calculate relative wavelet energy
    energies = []
    for c in coeffs:
        energies.append(np.sum(np.square(c)))

    decomp = ['A5', 'D1', 'D2', 'D3', 'D4', 'D5']

    energies[1:6] = energies[-1:-6:-1]

    total_energy = np.sum(energies)
    relative_energies = [(e / total_energy) * 100 for e in energies]

    # print(relative_energies)

    # plt.plot(decomp, energies)
    # plt.xlabel('Dimension Number')
    # plt.ylabel('Wavelet Bispectrum Energy')
    # plt.show()

    # plt.plot(decomp, relative_energies)
    # plt.xlabel('Dimension Number')
    # plt.ylabel('Relative Wavelet Bispectrum Energy')
    # plt.show()

    return relative_energies

### Bispectrum 2d 25 x 1

In [24]:
def extract_bispec2dFlat(directory, lag, filter_dim = 5):
    if 'test' in directory.lower():
        dest_dir = os.path.join('../Feature','Bispectrum2DFlat','Test')
    else:
        dest_dir = os.path.join('../Feature','Bispectrum2DFlat','Train')
    bis_dim = lag
    big_fil_size = bis_dim //2
    sigma = bis_dim//8
    if sigma == 0:
        sigma = 1
    fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
    
    if sum(fil_size) != bis_dim:
            fil_size = [value for value in fil_size if value != 0]
            if len(fil_size) == 1:
                fil_size.append(fil_size[0])
    
    fil_size[-1] = fil_size[-2] 
    pyramid = np.zeros([bis_dim,bis_dim])
    xtrack, ytrack = 0,0
    print(fil_size)
    for xdim in range(len(fil_size)):
        for ydim in range(len(fil_size)):
            x,y = fil_size[xdim], fil_size[ydim]
            pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = gkernel(x, y, sigma)
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    recap = pd.DataFrame(columns=['Wall Time', 'CPU Time'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(directory.replace('CSV', 'bispectrum2dFlat')[3:]+"_" + str(lag),foldername).lower()
            des_dir = os.path.join(dest_dir, des_dir)
            files = os.listdir(folder)
            for filename in files:
                cpu_start = time.process_time()
                wt_start = time.time()
                if filename in zeros_train or filename in zeros_test:
                    continue
                rel_path = os.path.join(directory, foldername, filename)
                if 'metadata' in filename.lower():
                    continue
                trial_number = filename.split('.')[0].split('_')[1]
                des_file = foldername+'_'+ str(trial_number) + '_bispectrum2dFlat' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                df_data, channel_name = get_csv_EEG(rel_path)
                bispectrum = []
                for channel in channel_name:
                    magnitude = calcBispecMag(df_data[channel], t, lag)
                    magnitude = magnitude[:256, :256]
                    feature = create_feature(magnitude,pyramid,fil_size)
                    bispectrum.append(feature.flatten())
                bispectrum = np.array(bispectrum)
                np.save(des_path, bispectrum)
                wt_end = time.time()
                cpu_end = time.process_time()
                wall_time = wt_end - wt_start
                cpu_time = cpu_end - cpu_start
                recap_temp = pd.DataFrame([[wall_time, cpu_time]],columns=recap.columns)
                recap = pd.concat([recap, recap_temp], ignore_index=True)
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
    recap_dir = os.path.join('./logs/Execution',directory.split('/')[1])
    if not os.path.exists(recap_dir):
        os.makedirs(recap_dir)
    recap_path = os.path.join(recap_dir,'recap_bispectrum2dFlat'+str(lag)+'.csv')
    recap.to_csv(recap_path)    


### Bispectrum 2d 5 x 5

In [20]:
def extract_bispec2d(directory, lag, filter_dim = 5):
    if 'test' in directory.lower():
        dest_dir = os.path.join('../Feature','Bispectrum2D','Test')
    else:
        dest_dir = os.path.join('../Feature','Bispectrum2D','Train')
    bis_dim = lag
    big_fil_size = bis_dim //2
    sigma = bis_dim//8
    if sigma == 0:
        sigma = 1
    fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
    
    if sum(fil_size) != bis_dim:
            fil_size = [value for value in fil_size if value != 0]
            if len(fil_size) == 1:
                fil_size.append(fil_size[0])
    
    fil_size[-1] = fil_size[-2] 
    pyramid = np.zeros([bis_dim,bis_dim])
    xtrack, ytrack = 0,0
    print(fil_size)
    for xdim in range(len(fil_size)):
        for ydim in range(len(fil_size)):
            x,y = fil_size[xdim], fil_size[ydim]
            pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = gkernel(x, y, sigma)
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    recap = pd.DataFrame(columns=['Wall Time', 'CPU Time'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(directory.replace('CSV', 'bispectrum2d')[3:]+"_" + str(lag),foldername).lower()
            des_dir = os.path.join(dest_dir, des_dir)
            files = os.listdir(folder)
            for filename in files:
                cpu_start = time.process_time()
                wt_start = time.time()
                if filename in zeros_train or filename in zeros_test:
                    continue
                rel_path = os.path.join(directory, foldername, filename)
                if 'metadata' in filename.lower():
                    continue
                trial_number = filename.split('.')[0].split('_')[1]
                des_file = foldername+'_'+ str(trial_number) + '_bispectrum2d' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                df_data, channel_name = get_csv_EEG(rel_path)
                bispectrum = []
                for channel in channel_name:
                    magnitude = calcBispecMag(df_data[channel], t, lag)
                    magnitude = magnitude[:256, :256]
                    feature = create_feature(magnitude,pyramid,fil_size)
                    bispectrum.append(feature)
                bispectrum = np.array(bispectrum)
                print(des_path)
                np.save(des_path, bispectrum)
                wt_end = time.time()
                cpu_end = time.process_time()
                wall_time = wt_end - wt_start
                cpu_time = cpu_end - cpu_start
                recap_temp = pd.DataFrame([[wall_time, cpu_time]],columns=recap.columns)
                recap = pd.concat([recap, recap_temp], ignore_index=True)
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
    recap_dir = os.path.join('./logs/Execution',directory.split('/')[1])
    if not os.path.exists(recap_dir):
        os.makedirs(recap_dir)
    recap_path = os.path.join(recap_dir,'recap_bispectrum2d'+str(lag)+'.csv')
    recap.to_csv(recap_path)    


### RWB 2D

In [25]:
def extract_rwb2d(directory, lag):
    if 'test' in directory.lower():
        dest_dir = os.path.join('../Feature','RWB2D','Test')
    else:
        dest_dir = os.path.join('../Feature','RWB2D','Train')
    recap = pd.DataFrame(columns=['Wall Time', 'CPU Time'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(directory.replace('CSV', 'rwb2d')[3:]+"_" + str(lag),foldername).lower()
            des_dir = os.path.join(dest_dir, des_dir)
            files = os.listdir(folder)
            for filename in files:
                cpu_start = time.process_time()
                wt_start = time.time()
                if filename in zeros_train or filename in zeros_test:
                    continue
                rel_path = os.path.join(directory, foldername, filename)
                if 'metadata' in filename.lower():
                    continue
                trial_number = filename.split('.')[0].split('_')[1]
                df_data, channel_name = get_csv_EEG(rel_path)
                des_file = foldername+'_'+ str(trial_number) + '_rwb2d' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                RWB = []
                for channel in channel_name:
                    y = df_data[channel]; # sinyal per channel
                    # N = len(y);
                    # z = y - np.mean(y);
                    # nsamp = len (y[0])
                    relative_energies = calcRelativeEnergy(calcWaveletDec(calcCumulantOrde3(y, t, lag)), y)
                    RWB.append(relative_energies)
                RWB = np.array(RWB)
                np.save(des_path, RWB)
                wt_end = time.time()
                cpu_end = time.process_time()
                wall_time = wt_end - wt_start
                cpu_time = cpu_end - cpu_start
                recap_temp = pd.DataFrame([[wall_time, cpu_time]],columns=recap.columns)
                recap = pd.concat([recap, recap_temp], ignore_index=True)
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
    recap_dir = os.path.join('./logs/Execution',directory.split('/')[1])
    if not os.path.exists(recap_dir):
        os.makedirs(recap_dir)
    recap_path = os.path.join(recap_dir,'recap_rwb2d'+str(lag)+'.csv')
    recap.to_csv(recap_path)


### Bispectrum 1D

In [ ]:
def extract_bispec1d(directory, lag, filter_dim = 5):
    if 'test' in directory.lower():
        dest_dir = os.path.join('../Feature','Bispectrum1D','Test')
    else:
        dest_dir = os.path.join('../Feature','Bispectrum1D','Train')
    bis_dim = lag
    big_fil_size = bis_dim //2
    sigma = bis_dim//8
    if sigma == 0:
        sigma = 1
    fil_size = [big_fil_size // (2 ** i) for i in range(filter_dim)]
    
    if sum(fil_size) != bis_dim:
            fil_size = [value for value in fil_size if value != 0]
            if len(fil_size) == 1:
                fil_size.append(fil_size[0])
    
    fil_size[-1] = fil_size[-2] 
    pyramid = np.zeros([bis_dim,bis_dim])
    xtrack, ytrack = 0,0
    print(fil_size)
    for xdim in range(len(fil_size)):
        for ydim in range(len(fil_size)):
            x,y = fil_size[xdim], fil_size[ydim]
            pyramid[xtrack:xtrack+x, ytrack:ytrack+y] = gkernel(x, y, sigma)
            ytrack = ytrack+y
        ytrack = 0
        xtrack = xtrack+x
    recap = pd.DataFrame(columns=['Wall Time', 'CPU Time'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(directory.replace('CSV', 'bispectrum1d')[3:]+"_" + str(lag),foldername).lower()
            des_dir = os.path.join(dest_dir, des_dir)
            files = os.listdir(folder)
            for filename in files:
                cpu_start = time.process_time()
                wt_start = time.time()
                if filename in zeros_train or filename in zeros_test:
                    continue
                rel_path = os.path.join(directory, foldername, filename)
                if 'metadata' in filename.lower():
                    continue
                trial_number = filename.split('.')[0].split('_')[1]
                des_file = foldername+'_'+ str(trial_number) + '_bispectrum1d' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                df_data, channel_name = get_csv_EEG(rel_path)
                bispectrum = []
                for channel in channel_name:
                    magnitude = calcBispecMag(df_data[channel], t, lag)
                    magnitude = magnitude[:256, :256]
                    feature = create_feature(magnitude,pyramid,fil_size)
                    bispectrum.append(feature.flatten())
                bispectrum = np.array(bispectrum)
                bispectrum = bispectrum.flatten()
                np.save(des_path, bispectrum)
                wt_end = time.time()
                cpu_end = time.process_time()
                wall_time = wt_end - wt_start
                cpu_time = cpu_end - cpu_start
                recap_temp = pd.DataFrame([[wall_time, cpu_time]],columns=recap.columns)
                recap = pd.concat([recap, recap_temp], ignore_index=True)
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
    recap_dir = os.path.join('./logs/Execution',directory.split('/')[1])
    if not os.path.exists(recap_dir):
        os.makedirs(recap_dir)
    recap_path = os.path.join(recap_dir,'recap_bispectrum1d'+str(lag)+'.csv')
    recap.to_csv(recap_path)    


### Main Program

In [27]:
lags = [256, 128, 64, 32, 16, 8, 4, 2]
for lag in lags:
    extract_rwb2d('../SMNI_CMI_TEST_CSV', lag)
    extract_bispec2dFlat('../SMNI_CMI_TEST_CSV', lag)
    extract_bispec2d('../SMNI_CMI_TEST_CSV', lag)
    extract_bispec2d('../SMNI_CMI_TRAIN_CSV', lag)
    extract_bispec2dFlat('../SMNI_CMI_TRAIN_CSV', lag)
    extract_rwb2d('../SMNI_CMI_TRAIN_CSV', lag)

KeyboardInterrupt: 